# All companies creation by postal code

In [10]:
%run "../config/notebook.ipynb"
%run "../config/files.ipynb"

# load preparation from cache
FROM_CACHE=True

# import communes of the Haute-Garonne: will be imported while importing sirenes
# import the Insee staffing ranges
%run "../prepare/prepare_insee_staffing_ranges.ipynb"
# import Haute-Garonne sirenes
%run "../prepare/prepare_companies_sirene_registry.ipynb"
# import Haute-garonne map
%run "../prepare/prepare_map_hautegaronne_postalcodes.ipynb"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from matplotlib.patches import Patch

%matplotlib inline

(from cache) df_staffing_ranges: 15 staffing ranges


(from cache) df_communes_haute_garonne: 586 communes
(from cache) df_sirenes: 598958 companies


(from cache) gdf_hautegaronne_postalcodes: 70 postalcodes


## Create the companies creatio  by postalcode dataframe 

In [11]:
df_sirenes.head()

,siren,siret,creation_date,staffing_range,postalcode,year,nom
0,213100019,21310001900016,1983-03-01,02,31230,1983.0,Agassac
1,213100019,21310001900024,1983-12-21,01,31230,1983.0,Agassac
2,263101404,26310140400014,1984-06-19,NN,31230,1984.0,Agassac
3,263108763,26310876300016,1984-11-20,NN,31230,1984.0,Agassac
4,309167526,30916752600016,1977-01-01,01,31230,1977.0,Agassac


In [15]:
companies_by_postalcode = df_sirenes.groupby('postalcode')['postalcode'].count()

def classify_creation(value):
    target = value // 10000
    return target

companies_creationclass_by_postalcode = companies_by_postalcode.apply(classify_creation)

## Create the geodataframe

In [16]:
gdf_hautegaronne_postalcodes_companies_creation = gdf_hautegaronne_postalcodes.merge(companies_creationclass_by_postalcode, left_on='ID', right_on='postalcode')

ValueError: 'postalcode' is both an index level and a column label, which is ambiguous.